In [1]:
from openai import OpenAI,AsyncOpenAI
client = OpenAI(base_url="https://api.302.ai/v1/chat/completions", api_key="sk-gPl43py4bj6TN3mksIN18FPbRZXdHoV6i0XOAAshKFpanIb1")#ollama服务默认端口


In [45]:
# def get_weahter(location):
#    if location=="Shanghai":
#        return "sunny"
#    elif location=="Beijing":
#        return "rainy"
#    else:
#        return f"I don't know the weahter of location {location}"
def get_weahter(location,food):
    if location == "Shanghai":
        return f"It's sunny in {location} with {food}"
    elif location == "Beijing":
        return f"It's rainy in {location} with {food}."
    else:
        return f"I don't know the weather of {location} with {food}."

#定义第二个工具的函数实现,该工具能够实现输入一个地名，返回一句话我已经订好票了并且在周四的时候去这个地方玩
def get_visiting_place(visiting_place):
    return f'I have order the ticket and we will go {visiting_place} on Thursday.'
#定义大模型调用工具的schema(工具的定义)
tools=[{
        "type": "function",
            "function": {
                "name": "get_weather",
                "description": "获取当前位置的天气和当地的特色食物",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "中国的省份名称的拼音表示,第一个字大写如:Sichuan",
                            "default":"未提及"
                        },
                        "food": {
                            "type": "string",
                            "description": "根据当地的特色食物输出",
                            "default":"未提及"
                        },
                    }
                },
                "required": ["location"] #必填参数
            }
    },
    {
        "type": "function",
            "function": {
            "name": "get_visiting_place",
            "description": "获取去访问的景点",
            "parameters": {
                "type": "object",
                "properties": {
                    "visiting_place": {
                        "type": "string",
                        "description": "景点名称",
                    },
                }
            },
            "required": ["visiting_place"]  # 必填参数
        },
    }]

In [46]:
import json
response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        # {"role": "user", "content": "你好"},
        {"role": "user", "content": "我想去北京玩,请问北京的天气如何？我想去故宫玩"},
    ],
    stream=False, #非流式输出
    tools=tools
)
check_functions =  {"get_weather":get_weahter,
                    "get_visiting_place":get_visiting_place}
print(response)
if response.choices[0].finish_reason=="tool_calls": #通过finish_reason确认是哪一类回复
    #遍历每一个工具中提取出来的槽位参数
    for trigger_function in response.choices[0].message.tool_calls:
        print('---',trigger_function)
        function_name=trigger_function.function.name
        function_args=json.loads(trigger_function.function.arguments) #'{"location": "Beijing"}'->{"location": "Beijing","food":"豆浆"}
        #工具调用函数实现结果(通过工具字典及工具名称找到对应的工具然后调用):
        function_to_call = check_functions[function_name]
        #传入参数得到工具调用结果
        function_result=function_to_call(**function_args) #location="Beijing"，food='豆浆'
        print("调用的工具:",function_name,"槽位参数:",function_args,"工具调用结果:",function_result)
else:     #没有触发工具调用
    return_message=response.choices[0].message.content
    print("未触发工具调用，直接回复:",return_message)

ChatCompletion(id='chatcmpl-Br9excqJX8QeGJLOJI28kRFUVUoye', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_n3c4TcCJQTEMCqLDf390KgeB', function=Function(arguments='{"location": "Beijing"}', name='get_weather'), type='function'), ChatCompletionMessageToolCall(id='call_XKXtOIFhftmM049KiazfD6VE', function=Function(arguments='{"visiting_place": "故宫"}', name='get_visiting_place'), type='function')]))], created=1752008215, model='gpt-4.1', object='chat.completion', service_tier=None, system_fingerprint='fp_07e970ab25', usage=CompletionUsage(completion_tokens=51, prompt_tokens=152, total_tokens=203, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, 

TypeError: get_weahter() missing 1 required positional argument: 'food'

In [48]:
from openai import OpenAI,AsyncOpenAI #导入异步客户端库
deepseek_async_client = AsyncOpenAI(api_key="sk-gPl43py4bj6TN3mksIN18FPbRZXdHoV6i0XOAAshKFpanIb1", base_url="https://api.302.ai/v1/chat/completions")

async def deepseek_steam_llm(tools,messages):
    # print(tools,messages)
    stream =await deepseek_async_client.chat.completions.create(
        model="gpt-4.1",
        messages=messages,
        stream=True, #流式输出
        tools=tools
    )
    async for chunk in stream:
        # print("chunk:", chunk)
        yield chunk #注意使用yield输出,不能用return
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    # {"role": "user", "content": "你好"},
    {"role": "user", "content": "我想去北京玩,请问北京的天气如何？我想去故宫玩"},
]      
tool_calls_dict={}
async for chunk in deepseek_steam_llm(tools, messages):
    if chunk.choices: #元数据不要
        if chunk.choices[0].delta.tool_calls:
            for tool_call in chunk.choices[0].delta.tool_calls:
                tool_call_id = tool_call.id
                function_name = tool_call.function.name
                arguments_fragment = tool_call.function.arguments
                # print("arguments_fragment:", arguments_fragment)
                # 初始化工具调用信息 tool_call_id为None的话也不开一个新的key
                if tool_call_id not in tool_calls_dict and tool_call_id:
                    tool_calls_dict[tool_call_id] = {
                        "name": function_name,
                        "arguments": ""
                    }
                 # 拼接参数片段n
                if tool_call_id and arguments_fragment:
                    tool_calls_dict[tool_call_id]["arguments"] += arguments_fragment
                #此时arguments_fragment为None
                elif not arguments_fragment:
                    continue
                else:
                    # 获取最后一个有key的值,
                    last_key_id = list(tool_calls_dict.keys())[-1]
                    tool_calls_dict[last_key_id]["arguments"] += arguments_fragment
                print("触发functioncalling回复:", tool_calls_dict)
        if chunk.choices[0].delta.content:  # 检查是否有内容决定是function call还是不同回复
               
                print("触发普通问答回复:",chunk.choices[0].delta.content)
        if chunk.choices[0].finish_reason == "tool_calls":
                print("工具调用结束")
    
# async  def main():
#     messages = [
#         {"role": "system", "content": "You are a helpful assistant"},
#         # {"role": "user", "content": "你好"},
#         {"role": "user", "content": "我想去北京玩,请问北京的天气如何？我想去故宫玩"},
#     ]
#     #流式输出
#     async for chunk in deepseek_steam_llm(tools, messages):
#         print(chunk)
# 
# # 然后你需要运行这个异步函数，例如：
# import asyncio
# asyncio.run(main())

触发functioncalling回复: {'call_7Udfcz28u9cBucH7zMCxUUPV': {'name': 'get_weather', 'arguments': '{"lo'}}
触发functioncalling回复: {'call_7Udfcz28u9cBucH7zMCxUUPV': {'name': 'get_weather', 'arguments': '{"locatio'}}
触发functioncalling回复: {'call_7Udfcz28u9cBucH7zMCxUUPV': {'name': 'get_weather', 'arguments': '{"location": "B'}}
触发functioncalling回复: {'call_7Udfcz28u9cBucH7zMCxUUPV': {'name': 'get_weather', 'arguments': '{"location": "Beiji'}}
触发functioncalling回复: {'call_7Udfcz28u9cBucH7zMCxUUPV': {'name': 'get_weather', 'arguments': '{"location": "Beijing"}'}}
触发functioncalling回复: {'call_7Udfcz28u9cBucH7zMCxUUPV': {'name': 'get_weather', 'arguments': '{"location": "Beijing"}'}, 'call_78HqeNfkCCuSWrUDW09eFIG7': {'name': 'get_visiting_place', 'arguments': '{"vi'}}
触发functioncalling回复: {'call_7Udfcz28u9cBucH7zMCxUUPV': {'name': 'get_weather', 'arguments': '{"location": "Beijing"}'}, 'call_78HqeNfkCCuSWrUDW09eFIG7': {'name': 'get_visiting_place', 'arguments': '{"visitin'}}
触发functioncalling回复: {'call_

In [8]:

deepseek_client = AsyncOpenAI(api_key="sk-gPl43py4bj6TN3mksIN18FPbRZXdHoV6i0XOAAshKFpanIb1", base_url="https://api.302.ai/v1/chat/completions")
messages_ = [{'role': 'system', 'content': '我是留学助手，根据以下内容推荐合适的学校以及入学时间和金额'}, {'role': 'user', 'content': '我想申请本科的哈佛大学;已知道的信息intention_school:哈佛大学academic_degree:本科current_school:浙江宁波大学gpa_point:4.0'}]
m=[
        {"role": "system", "content": "You are a helpful assistant"},
        # {"role": "user", "content": "你好"},
        {"role": "user", "content": "我想去北京玩,请问北京的天气如何？我想去故宫玩"},
    ],
# stream = deepseek_client.chat.completions.create(
#     model="gpt-4.1", #gpt-4.1
#     messages=m,
#     # max_tokens=8000,
#     # temperature=0,
#     stream=True,
# )
# for chunk in stream:
#     print("chunk:", chunk)
response = client.chat.completions.create(
model="gpt-4.1",
messages=messages_,
stream=False, #非流式输出
) 
for chunk in response:
    print("chunk:", chunk.choices)

chunk: ('id', 'chatcmpl-BrLJU0d7UBJG1bieCJE27vDZXhwKb')
chunk: ('choices', [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='根据你的信息：\n\n- 目标学校（intention_school）：哈佛大学（Harvard University）\n- 申请学位（academic_degree）：本科\n- 当前学校（current_school）：浙江宁波大学（可能为本科在读，说明你计划转学或是大一时申请重读本科）\n- GPA：4.0（满分）\n\n### 一、哈佛大学本科申请建议\n\n#### 1. 你能否申请哈佛本科？\n- 一般来说，哈佛大学本科只接受高中毕业生，全球本科转学生录取极为有限，且竞争极为激烈。你现在在宁波大学读本科，如果是大一，可以申请哈佛大一阶段或者转学（Transfer）。如果已经超过大一，则只可作为转学生申请。\n- 哈佛转学通常要求：\n  - 大一/大二在读（即累计不超过两年大学课程）\n  - 拥有优异的大学成绩和高中记录\n  - 优秀的托福/雅思/SAT/ACT成绩\n  - 活跃的课外活动、社会实践、领导力经历\n\n#### 2. 入学时间\n- 哈佛本科通常秋季入学（8月份）。\n- 通常在每年3月1日截止转学申请（Transfer Application Deadline），入学时间为同年8-9月新学年。\n- 如果是重读申请（Freshman application），截止时间为每年1月1日。\n\n#### 3. 申请所需金额/费用\n- 哈佛大学2024-2025学年本科学杂费和生活费总计约为$83,538美元/年，包含：\n  - 学费：$59,360\n  - 住宿和食宿：$21,420\n  - 其他费用（书本、个人开销等）：约$2,758\n- 哈佛有非常慷慨的need-based Financial Aid，如果你家庭年收入低于$85,000，可能全额资助学费和生活费。根据家庭经济情况不同，绝大多数国际生能获得一定比例的奖助学金。\n\n#### 4. 其他核心要求\n- 英语成绩：TOEFL 

In [43]:
for chunk in response:
    a=chunk
    if a[0]=='choices':
        result=a[1][0].message.content
        print(result)

根据你的信息：

- 目标学校（intention_school）：哈佛大学（Harvard University）
- 申请学位（academic_degree）：本科
- 当前学校（current_school）：浙江宁波大学（可能为本科在读，说明你计划转学或是大一时申请重读本科）
- GPA：4.0（满分）

### 一、哈佛大学本科申请建议

#### 1. 你能否申请哈佛本科？
- 一般来说，哈佛大学本科只接受高中毕业生，全球本科转学生录取极为有限，且竞争极为激烈。你现在在宁波大学读本科，如果是大一，可以申请哈佛大一阶段或者转学（Transfer）。如果已经超过大一，则只可作为转学生申请。
- 哈佛转学通常要求：
  - 大一/大二在读（即累计不超过两年大学课程）
  - 拥有优异的大学成绩和高中记录
  - 优秀的托福/雅思/SAT/ACT成绩
  - 活跃的课外活动、社会实践、领导力经历

#### 2. 入学时间
- 哈佛本科通常秋季入学（8月份）。
- 通常在每年3月1日截止转学申请（Transfer Application Deadline），入学时间为同年8-9月新学年。
- 如果是重读申请（Freshman application），截止时间为每年1月1日。

#### 3. 申请所需金额/费用
- 哈佛大学2024-2025学年本科学杂费和生活费总计约为$83,538美元/年，包含：
  - 学费：$59,360
  - 住宿和食宿：$21,420
  - 其他费用（书本、个人开销等）：约$2,758
- 哈佛有非常慷慨的need-based Financial Aid，如果你家庭年收入低于$85,000，可能全额资助学费和生活费。根据家庭经济情况不同，绝大多数国际生能获得一定比例的奖助学金。

#### 4. 其他核心要求
- 英语成绩：TOEFL iBT≥100或雅思≥7.5
- 标化成绩：需要提交SAT/ACT（哈佛2024-2025申请周期恢复标化成绩要求）
- 推荐信/高中及大学成绩单/课外活动及文书

---

### 二、申请方案建议

| 申请方向         | 是否建议         | 具体要求                       | 申请截止时间     | 入学时间       | 预计总费用/年  